In [184]:
#!/usr/bin/env python
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# tsv_file='TH_data_challenge.tsv'
# csv_table=pd.read_table(tsv_file,sep='\t')
# csv_table.to_csv('data_.csv',index=False)

In [185]:
df = pd.read_csv('data_.csv', sep=',')

In [186]:
Y=df.loc[:, :'dim_is_requested']
df.drop(df.columns[[0,1,2,3,4]], axis=1, inplace=True)
#print df.head()

In [187]:
def myfillna(series):
    if series.dtype is pd.np.dtype(float):
        #print ("found")
        return series.fillna(-1)
    elif series.dtype is pd.np.dtype(object):
        #print ("case2")
        return series.fillna('.')
    else:
        return series

In [188]:
df = df.apply(myfillna)
#print df.head()
print df.dtypes

m_effective_daily_price                               float64
m_pricing_cleaning_fee                                float64
dim_market                                             object
dim_lat                                               float64
dim_lng                                               float64
dim_room_type                                          object
dim_person_capacity                                     int64
dim_is_instant_bookable                                  bool
m_checkouts                                           float64
m_reviews                                             float64
days_since_last_booking                               float64
cancel_policy                                           int64
image_quality_score                                   float64
m_total_overall_rating                                float64
m_professional_pictures                               float64
dim_has_wireless_internet                               int64
ds_night

In [189]:
X = df.select_dtypes(include=[object,bool])
X_other= df.select_dtypes(include=[float,int])
print X.head(3)
print X.shape
print X_other.shape

    dim_market dim_room_type  dim_is_instant_bookable
0  Los Angeles  Private room                    False
1  Los Angeles  Private room                     True
2  Los Angeles  Private room                    False
(184279, 3)
(184279, 38)


In [190]:
from sklearn import preprocessing
print X.columns
# TODO: create a LabelEncoder object and fit it to each feature in X
# 1. INSTANTIATE
# encode labels with value between 0 and n_classes-1.
le = preprocessing.LabelEncoder()
# 2/3. FIT AND TRANSFORM
# use df.apply() to apply le.fit_transform to all columns
X_2 = X.apply(le.fit_transform)
X_2.head(10)

Index([u'dim_market', u'dim_room_type', u'dim_is_instant_bookable'], dtype='object')


,dim_market,dim_room_type,dim_is_instant_bookable
0,0,1,0
1,0,1,1
2,0,1,0
3,0,0,0
4,0,0,0
5,0,1,0
6,0,1,0
7,0,0,0
8,0,2,0
9,0,0,0


In [191]:
# 1. INSTANTIATE
enc = preprocessing.OneHotEncoder()
# 2. FIT
enc.fit(X_2)
# 3. Transform
onehotlabels = enc.transform(X_2).toarray()
print onehotlabels.shape
print onehotlabels

(184279, 8)
[[1. 0. 0. ... 0. 1. 0.]
 [1. 0. 0. ... 0. 0. 1.]
 [1. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 1. ... 1. 1. 0.]
 [0. 0. 1. ... 0. 0. 1.]
 [0. 0. 1. ... 0. 1. 0.]]


In [192]:
X_=np.concatenate((onehotlabels, X_other), axis=1)
print X_.shape

(184279, 46)


In [193]:
from sklearn.preprocessing import LabelEncoder

#Auto encodes any dataframe column of type category or object.
def dummyEncode(df):
        columnsToEncode = list(df.select_dtypes(include=['category','object','bool']))
        le = LabelEncoder()
        for feature in columnsToEncode:
            try:
                df[feature] = le.fit_transform(df[feature])
            except:
                print('Error encoding '+feature)
        return df

In [194]:
Y=dummyEncode(Y)
Y=Y.as_matrix()
print Y[0]

[0]


In [195]:
# def indices_to_one_hot(data, nb_classes):
#     """Convert an iterable of indices to one-hot encoded labels."""
#     targets = np.array(data).reshape(-1)
#     return np.eye(nb_classes)[targets]

# Y=indices_to_one_hot(2, Y)
# print Y
print X_[0]
print Y[0]

[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  1.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
  1.10000000e+02  6.00000000e+01  3.40539320e+01 -1.18362970e+02
  2.00000000e+00  2.40000000e+01  1.90000000e+01  3.20000000e+02
  4.00000000e+00  8.30959000e-01  7.40000000e+01  1.60000000e+01
  1.00000000e+00  2.00000000e+00  6.00000000e+00  0.00000000e+00
  7.00000000e+00 -1.00000000e+00 -1.00000000e+00  0.00000000e+00
  5.00000000e+00  9.00000000e+01  9.20000000e+01 -1.00000000e+00
 -1.00000000e+00  1.33333333e-01  7.88142857e+02  2.41428571e+02
  4.14285714e+01  1.00000000e+00  9.84000000e+03  1.31428576e+00
  1.79591837e+00  5.70000000e+01  4.80000000e+01  7.90000000e+01
  7.50000000e+01  9.80000000e+01]
[0]


In [196]:
from sklearn.model_selection import train_test_split
import warnings
import operator
warnings.filterwarnings("ignore")
X_train, X_test, y_train, y_test = train_test_split(X_, Y, test_size=0.3, random_state=42)

In [197]:
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
import warnings
import heapq
import time
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [198]:
# clf = RandomForestClassifier(n_estimators=1000,bootstrap=True,min_samples_leaf=10,min_samples_split=2,criterion='gini',max_features=3, max_depth=None)
# clf=clf.fit(X_train,y_train)
# print(clf.feature_importances_)
# y_pred=clf.predict(X_test)
# from sklearn.metrics import accuracy_score
# print accuracy_score(y_test, y_pred)


In [199]:
# fit model no training data
# from xgboost import XGBClassifier
# model = XGBClassifier(learning_rate=0.1, n_estimators=1000, max_depth=5, min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8, objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27)
# model= model.fit(X_train, y_train)
# print(model.feature_importances_)
# y_pred=model.predict(X_test)
# print accuracy_score(y_test, y_pred)

In [200]:
# from sklearn import ensemble
# original_params = {'n_estimators': 1000, 'max_leaf_nodes': 4, 'max_depth': None, 'random_state': 2,
#                    'min_samples_split': 5}
# params = dict(original_params)
# clf = ensemble.GradientBoostingClassifier(**params)
# clf.fit(X_train, y_train)

In [201]:
# y_pred=model.predict(X_test)
# print accuracy_score(y_test, y_pred)

In [202]:
#print(clf.feature_importances_)

In [203]:
# import xgboost as xgb
# n_folds = 5
# early_stopping = 10
# params = {'eta': 0.02, 'max_depth': 5, 'subsample': 0.7, 'colsample_bytree': 0.7, 'objective': 'binary:logistic', 'seed': 99, 'silent': 1, 'eval_metric':'auc', 'nthread':4}

# xg_train = xgb.DMatrix(X_train, label=y_train);

#cv = xgb.cv(params, xg_train, 5000, nfold=n_folds, early_stopping_rounds=early_stopping, verbose_eval=1)


In [204]:
#print cv.params

In [205]:
# y_pred=cv.predict(X_test)
# print accuracy_score(y_test, y_pred)

In [206]:
#xg_train.save_binary('train.buffer')

In [207]:
#xgb.save_model('0001.model')

In [208]:
t=5000
params = {'n_estimators':5000, 'subsample': 0.7, 'colsample_bytree': 0.7, 'objective': 'binary:logistic', 'seed': 99, 'silent': 1, 'eval_metric':'auc', 'nthread':4}
#X_train, y_train=X_train[:t], y_train[:t]

In [ ]:
from sklearn.model_selection import GridSearchCV
cv=5
params_sklearn={}
param_untuned = {'booster':('gbtree', 'gblinear','dart'), 'max_depth':[2,5,7,10],'eta':[0.05,0.02,0.1,0.5,0.7]}
estimator = xgb.XGBClassifier(**params)
grid_search = GridSearchCV(estimator, param_grid=param_untuned, scoring='accuracy', cv=cv, verbose=1)
grid_search.fit(X_train, y_train)
df = pd.DataFrame(grid_search.cv_results_)[['params', 'mean_train_score', 'mean_test_score']]
#print df
print 'the best_params : ', grid_search.best_params_
print 'the best_score  : ', grid_search.best_score_
for k,v in grid_search.best_params_.items():
    params_sklearn[k] = v

Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


In [ ]:
evaluated=grid_search.best_params_
params=dict(params.items() + evaluated.items())

In [ ]:
estimator = xgb.XGBClassifier(**params).fit(X_train,y_train)

In [ ]:
y_pred= estimator.predict(X_test)
print accuracy_score(y_test, y_pred)

In [ ]:
print df